# RAG Basique avec Qdrant et Mistral

Ce notebook implémente un système RAG minimal pour interroger des documents (CSV, TXT, JSON, PDF)

## 1. Imports et Configuration

In [1]:
import os
import json
import csv
from pathlib import Path
from dotenv import load_dotenv

from langchain_mistralai import MistralAIEmbeddings, ChatMistralAI
from langchain_qdrant import QdrantVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from qdrant_client import QdrantClient

from pypdf import PdfReader

# Charger les variables d'environnement
load_dotenv()

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
QDRANT_ENDPOINT = os.getenv("QDRANT_ENDPOINT")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

print("✓ Configuration chargée")

/home/user/code/claude/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


✓ Configuration chargée


## 2. Initialisation des Composants

In [2]:
# Initialiser le client Qdrant
qdrant_client = QdrantClient(
    url=QDRANT_ENDPOINT,
    api_key=QDRANT_API_KEY
)

# Initialiser les embeddings Mistral
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

# Initialiser le modèle de chat
llm = ChatMistralAI(
    model="mistral-small-latest",
    mistral_api_key=MISTRAL_API_KEY,
    temperature=0
)

COLLECTION_NAME = "documents_rag"

print("✓ Composants initialisés")

✓ Composants initialisés


## 3. Chargement des Documents

In [3]:
def load_txt(file_path):
    """Charger un fichier TXT"""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return [Document(page_content=content, metadata={"source": file_path, "type": "txt"})]

def load_json(file_path):
    """Charger un fichier JSON"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    content = json.dumps(data, indent=2, ensure_ascii=False)
    return [Document(page_content=content, metadata={"source": file_path, "type": "json"})]

def load_csv(file_path):
    """Charger un fichier CSV"""
    documents = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            content = "\n".join([f"{k}: {v}" for k, v in row.items()])
            documents.append(
                Document(page_content=content, metadata={"source": file_path, "type": "csv", "row": i})
            )
    return documents

def load_pdf(file_path):
    """Charger un fichier PDF"""
    reader = PdfReader(file_path)
    documents = []
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text.strip():
            documents.append(
                Document(page_content=text, metadata={"source": file_path, "type": "pdf", "page": i})
            )
    return documents

def load_documents_from_directory(directory):
    """Charger tous les documents d'un répertoire"""
    all_documents = []
    data_path = Path(directory)
    
    if not data_path.exists():
        print(f"Le dossier {directory} n'existe pas")
        return all_documents
    
    loaders = {
        '.txt': load_txt,
        '.json': load_json,
        '.csv': load_csv,
        '.pdf': load_pdf
    }
    
    for file_path in data_path.rglob('*'):
        if file_path.is_file():
            ext = file_path.suffix.lower()
            if ext in loaders:
                print(f"Chargement de {file_path.name}...")
                try:
                    docs = loaders[ext](str(file_path))
                    all_documents.extend(docs)
                except Exception as e:
                    print(f"Erreur lors du chargement de {file_path.name}: {e}")
    
    return all_documents

print("✓ Fonctions de chargement définies")

✓ Fonctions de chargement définies


## 4. Découpage et Indexation des Documents

In [4]:
# Charger les documents depuis le dossier data
documents = load_documents_from_directory("data")
print(f"✓ {len(documents)} documents chargés")

# Découper les documents en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

if documents:
    splits = text_splitter.split_documents(documents)
    print(f"✓ {len(splits)} chunks créés")
else:
    splits = []
    print("Aucun document à découper")

Chargement de exemple.txt...
Chargement de exemple.json...
Chargement de exemple.csv...
✓ 6 documents chargés
✓ 6 chunks créés


In [5]:
# Créer ou recréer la collection dans Qdrant
if splits:
    # Supprimer la collection si elle existe
    try:
        qdrant_client.delete_collection(COLLECTION_NAME)
        print(f"Collection {COLLECTION_NAME} supprimée")
    except:
        pass
    
    # Créer le vector store et indexer les documents
    vector_store = QdrantVectorStore.from_documents(
        splits,
        embeddings,
        url=QDRANT_ENDPOINT,
        api_key=QDRANT_API_KEY,
        collection_name=COLLECTION_NAME
    )
    
    print(f"✓ Documents indexés dans la collection '{COLLECTION_NAME}'")
else:
    print("Pas de documents à indexer")

Collection documents_rag supprimée
✓ Documents indexés dans la collection 'documents_rag'


## 5. Configuration du RAG

In [6]:
# Créer le retriever et la chaîne RAG
if splits:
    vector_store = QdrantVectorStore(
        client=qdrant_client,
        collection_name=COLLECTION_NAME,
        embedding=embeddings
    )
    
    retriever = vector_store.as_retriever(
        search_kwargs={"k": 3}
    )
    
    # Template pour le prompt RAG
    template = """Tu dois répondre UNIQUEMENT à partir des informations fournies dans le CONTEXTE ci-dessous.
Si une information n'est pas présente dans le CONTEXTE, dis clairement que tu ne sais pas.
Ne fabrique pas de réponses.

CONTEXTE:
{context}

QUESTION: {question}

RÉPONSE:"""

    prompt = ChatPromptTemplate.from_template(template)
    
    # Créer la chaîne RAG avec LCEL
    def format_docs(docs):
        return "\n\n".join([doc.page_content for doc in docs])
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    print("✓ Chaîne RAG configurée")
else:
    print("Impossible de configurer le RAG sans documents")

✓ Chaîne RAG configurée


## 6. Fonction de Requête

In [7]:
def query_rag(question):
    """Interroger le système RAG"""
    if not splits:
        return "Aucun document n'a été indexé. Veuillez ajouter des fichiers dans le dossier 'data'."
    
    # Récupérer les documents sources
    source_docs = retriever.invoke(question)
    
    # Obtenir la réponse
    answer = rag_chain.invoke(question)
    
    print("\n" + "="*80)
    print("QUESTION:")
    print(question)
    print("\n" + "-"*80)
    print("RÉPONSE:")
    print(answer)
    print("\n" + "-"*80)
    print("SOURCES:")
    for i, doc in enumerate(source_docs, 1):
        print(f"\n[{i}] {doc.metadata.get('source', 'N/A')} (type: {doc.metadata.get('type', 'N/A')})")
        print(f"    Extrait: {doc.page_content[:200]}...")
    print("="*80 + "\n")
    
    return {"answer": answer, "source_documents": source_docs}

print("✓ Fonction de requête définie")

✓ Fonction de requête définie


## 7. Exemples d'Utilisation

In [8]:
# Exemple de requête
if splits:
    result = query_rag("Résume les informations principales contenues dans les documents")
else:
    print("Ajoutez des fichiers dans le dossier 'data' et réexécutez les cellules 4 et 5")


QUESTION:
Résume les informations principales contenues dans les documents

--------------------------------------------------------------------------------
RÉPONSE:
Voici un résumé des informations principales contenues dans les documents :

1. **RAG (Retrieval-Augmented Generation)** :
   - Technique combinant recherche d'informations et génération de texte.
   - Avantages : réponses basées uniquement sur les informations fournies, réduction des hallucinations, travail avec des données privées, contexte actualisable sans réentraînement.

2. **Projet "RAG Académique"** :
   - Système RAG basique pour interroger des documents.
   - Technologies utilisées : Mistral (LLM), Qdrant (vectorstore), LangChain (orchestrateur), Streamlit (interface).
   - Formats supportés : CSV, TXT, JSON, PDF.
   - Caractéristiques : minimaliste, preuve de concept (POC), ne utilise pas Pandas.

3. **Streamlit** :
   - Rôle : Interface utilisateur.
   - Technologie : Framework web.
   - Description : Interfac

In [9]:
# Vous pouvez poser d'autres questions ici
# query_rag("Votre question")